<a href="https://colab.research.google.com/github/Ry02024/BookManager/blob/main/BookManager.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##環境構築

In [2]:
!pip install streamlit streamlit-webrtc pyzbar pillow requests opencv-python-headless pyngrok -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 773.8/773.8 kB 44.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 76.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 75.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 79.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 31.3 MB/s eta 0:00:00


In [3]:
!sudo apt-get install libzbar0

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  fonts-droid-fallback fonts-noto-mono fonts-urw-base35 ghostscript gsfonts
  imagemagick-6-common libdjvulibre-text libdjvulibre21 libfftw3-double3
  libgs9 libgs9-common libidn12 libijs-0.35 libjbig2dec0 libjxr-tools libjxr0
  liblqr-1-0 libmagickcore-6.q16-6 libmagickcore-6.q16-6-extra
  libmagickwand-6.q16-6 libv4l-0 libv4lconvert0 libwmflite-0.2-7 poppler-data
Suggested packages:
  fonts-noto fonts-freefont-otf | fonts-freefont-ttf fonts-texgyre
  ghostscript-x libfftw3-bin libfftw3-dev inkscape poppler-utils
  fonts-japanese-mincho | fonts-ipafont-mincho fonts-japanese-gothic
  | fonts-ipafont-gothic fonts-arphic-ukai fonts-arphic-uming fonts-nanum
The following NEW packages will be installed:
  fonts-droid-fallback fonts-noto-mono fonts-urw-base35 ghostscript gsfonts
  imagemagick-6-common libdjvulibre-text libdjvulibre21 libfftw3-

In [4]:
import streamlit as st
from pyngrok import ngrok

In [5]:
from google.colab import userdata
NGROK_AUTH_TOKEN = userdata.get('NGROK_AUTH_TOKEN')

In [6]:
from pyngrok import ngrok
import os

# ngrokトークンを設定
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

##実行ファイル

###version1

In [ ]:
%%writefile streamlit_app.py
import streamlit as st
from streamlit_webrtc import webrtc_streamer, VideoTransformerBase
import cv2
import numpy as np
from pyzbar.pyzbar import decode
import requests
from PIL import Image
from io import BytesIO

# APIから書籍データを取得する関数
def get_book_info(isbn):
    #url = f'https://www.googleapis.com/books/v1/volumes?q=isbn:{isbn}'
    url = f'https://openlibrary.org/api/books?bibkeys=ISBN:{isbn}&format=json&jscmd=data'
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if "items" in data:
            return data["items"][0]["volumeInfo"]
    return None

# 手動でISBN番号を入力する関数
def manual_isbn_input():
    isbn = st.text_input("ISBN番号を入力してください（例: 9784297108434）")
    if isbn:
        display_book_info(isbn)

# カメラでバーコードを読み取るクラス
class BarcodeScanner(VideoTransformerBase):
    def __init__(self):
        self.last_barcode = None

    def transform(self, frame):
        img = frame.to_ndarray(format="bgr24")
        decoded_objects = decode(img)

        for obj in decoded_objects:
            barcode_data = obj.data.decode("utf-8")
            barcode_type = obj.type

            # デバッグ: 認識したバーコード情報を表示
            st.write(f"認識したバーコード: {barcode_data} (タイプ: {barcode_type})")

            # ISBNは通常 "97" で始まる
            if barcode_type in ["EAN13", "ISBN13"] and barcode_data.startswith('97') and barcode_data != self.last_barcode:
                self.last_barcode = barcode_data
                st.session_state['barcode'] = barcode_data
                break

        return img

# カメラでバーコードを読み取る関数
def camera_isbn_input():
    if 'barcode' not in st.session_state:
        st.session_state['barcode'] = None
    webrtc_ctx = webrtc_streamer(key="barcode-scanner", video_transformer_factory=BarcodeScanner)
    if st.session_state['barcode']:
        barcode = st.session_state['barcode']
        st.success(f"バーコードが読み取られました: {barcode}")
        display_book_info(barcode)
    else:
        st.write("カメラをバーコードに向けてください。")

# 画像からバーコードを読み取る関数
def image_isbn_input():
    uploaded_file = st.file_uploader("バーコードを含む画像をアップロードしてください", type=["png", "jpg", "jpeg"])
    if uploaded_file is not None:
        image = Image.open(uploaded_file)
        st.image(image, caption='アップロードされた画像', use_column_width=True)
        img_array = np.array(image.convert('RGB'))
        decoded_objects = decode(img_array)
        for obj in decoded_objects:
            barcode_data = obj.data.decode("utf-8")
            barcode_type = obj.type
            if barcode_type in ["EAN13", "ISBN13"] and barcode_data.startswith('97'):
                st.session_state['barcode'] = barcode_data
                break
        if 'barcode' in st.session_state and st.session_state['barcode']:
            st.success(f"バーコードが読み取られました: {st.session_state['barcode']}")
            display_book_info(st.session_state['barcode'])
        else:
            st.error("バーコードが検出されませんでした。")

# 書籍情報を表示する関数
def display_book_info(isbn):
    book_info = get_book_info(isbn)
    if book_info:
        st.write("書籍情報:")
        st.write(f"**タイトル**: {book_info.get('title', '不明')}")
        st.write(f"**著者**: {', '.join(book_info.get('authors', ['不明']))}")
        st.write(f"**出版年**: {book_info.get('publishedDate', '不明')}")
        st.write(f"**出版社**: {book_info.get('publisher', '不明')}")
        st.write(f"**説明**: {book_info.get('description', '不明')}")
        if 'imageLinks' in book_info and 'thumbnail' in book_info['imageLinks']:
            image_url = book_info['imageLinks']['thumbnail']
            response = requests.get(image_url)
            img = Image.open(BytesIO(response.content))
            st.image(img, caption=book_info.get('title', '不明'))
    else:
        st.error("書籍情報が見つかりませんでした。")

st.title("ISBN番号で書籍データを検索するアプリ")

option = st.radio("選択してください:", ("ISBN番号を入力", "カメラでバーコードを読み取る", "画像からバーコードを読み取る"))

if option == "ISBN番号を入力":
    manual_isbn_input()
elif option == "カメラでバーコードを読み取る":
    camera_isbn_input()
elif option == "画像からバーコードを読み取る":
    image_isbn_input()

Writing streamlit_app.py


##version2

In [32]:

%%writefile streamlit_app.py
import streamlit as st
from streamlit_webrtc import webrtc_streamer, VideoTransformerBase
import cv2
import numpy as np
from pyzbar.pyzbar import decode
import requests
from PIL import Image
from io import BytesIO

# OpenBD APIから書籍データを取得する関数
def get_book_info_from_openbd(isbn):
    url = f'https://api.openbd.jp/v1/get?isbn={isbn}'
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data and data[0]:
            return data[0]['summary']
    return None

# 手動でISBN番号を入力する関数
def manual_isbn_input():
    isbn = st.text_input("ISBN番号を入力してください（例: 9784297108434）")
    st.write(f"入力されたISBN番号: {isbn}")  # デバッグ用
    if isbn:
        display_book_info(isbn)

# カメラでバーコードを読み取るクラス
class BarcodeScanner(VideoTransformerBase):
    def __init__(self):
        self.last_barcode = None

    def transform(self, frame):
        img = frame.to_ndarray(format="bgr24")
        decoded_objects = decode(img)

        for obj in decoded_objects:
            barcode_data = obj.data.decode("utf-8")
            barcode_type = obj.type

            # デバッグ: 認識したバーコード情報を表示
            st.write(f"認識したバーコード: {barcode_data} (タイプ: {barcode_type})")

            # ISBNは通常 "97" で始まる
            if barcode_type in ["EAN13", "ISBN13"] and barcode_data.startswith('97') and barcode_data != self.last_barcode:
                self.last_barcode = barcode_data
                st.session_state['barcode'] = barcode_data
                break

        return img

# カメラでバーコードを読み取る関数
def camera_isbn_input():
    if 'barcode' not in st.session_state:
        st.session_state['barcode'] = None
    webrtc_ctx = webrtc_streamer(key="barcode-scanner", video_transformer_factory=BarcodeScanner)
    if st.session_state['barcode']:
        barcode = st.session_state['barcode']
        st.success(f"バーコードが読み取られました: {barcode}")
        display_book_info(barcode)
    else:
        st.write("カメラをバーコードに向けてください。")

# 画像からバーコードを読み取る関数
def image_isbn_input():
    uploaded_file = st.file_uploader("バーコードを含む画像をアップロードしてください", type=["png", "jpg", "jpeg"])
    if uploaded_file is not None:
        image = Image.open(uploaded_file)
        st.image(image, caption='アップロードされた画像', use_column_width=True)
        img_array = np.array(image.convert('RGB'))
        decoded_objects = decode(img_array)
        st.write(f"デコードされたオブジェクト: {decoded_objects}")  # デバッグ用
        for obj in decoded_objects:
            barcode_data = obj.data.decode("utf-8")
            barcode_type = obj.type
            st.write(f"認識したバーコード: {barcode_data} (タイプ: {barcode_type})")  # デバッグ用
            if barcode_type in ["EAN13", "ISBN13"] and barcode_data.startswith('97'):
                st.session_state['barcode'] = barcode_data
                break
        if 'barcode' in st.session_state and st.session_state['barcode']:
            st.success(f"バーコードが読み取られました: {st.session_state['barcode']}")
            display_book_info(st.session_state['barcode'])
        else:
            st.error("バーコードが検出されませんでした。")

# 書籍情報を表示する関数
def display_book_info(isbn):
    book_info = get_book_info_from_openbd(isbn)
    if book_info:
        st.write("書籍情報:")
        st.write(f"**タイトル**: {book_info.get('title', '不明')}")
        st.write(f"**著者**: {book_info.get('author', '不明')}")
        st.write(f"**出版社**: {book_info.get('publisher', '不明')}")
        st.write(f"**出版年**: {book_info.get('pubdate', '不明')}")
        if 'cover' in book_info and book_info['cover']:
            image_url = book_info['cover']
            if image_url:  # URLが空でないことを確認
                response = requests.get(image_url)
                if response.status_code == 200:
                    img = Image.open(BytesIO(response.content))
                    st.image(img, caption=book_info.get('title', '不明'))
    else:
        st.error("書籍情報が見つかりませんでした。")

st.title("ISBN番号で書籍データを検索するアプリ")

option = st.radio("選択してください:", ("ISBN番号を入力", "カメラでバーコードを読み取る", "画像からバーコードを読み取る"))

if option == "ISBN番号を入力":
    manual_isbn_input()
elif option == "カメラでバーコードを読み取る":
    camera_isbn_input()
elif option == "画像からバーコードを読み取る":
    image_isbn_input()

Overwriting streamlit_app.py


##アプリの起動

In [33]:
# ngrokを介してStreamlitを公開
public_url = ngrok.connect(addr='8501')
print('Public URL:', public_url)

Public URL: NgrokTunnel: "https://8be0-34-34-63-127.ngrok-free.app" -> "http://localhost:8501"


In [34]:
# Streamlitアプリケーションの起動
!streamlit run streamlit_app.py >/dev/null

/content/streamlit_app.py:55: DeprecationWarning: The argument video_transformer_factory is deprecated. Use video_processor_factory instead.
See https://github.com/whitphx/streamlit-webrtc#for-users-since-versions-020
  webrtc_ctx = webrtc_streamer(key="barcode-scanner", video_transformer_factory=BarcodeScanner)
/content/streamlit_app.py:55: DeprecationWarning: The argument video_transformer_factory is deprecated. Use video_processor_factory instead.
See https://github.com/whitphx/streamlit-webrtc#for-users-since-versions-020
  webrtc_ctx = webrtc_streamer(key="barcode-scanner", video_transformer_factory=BarcodeScanner)


In [27]:
# APIから書籍データを取得する関数
def get_book_info(isbn):
    url = f'https://api.openbd.jp/v1/get?isbn={isbn}'
    response = requests.get(url)
    print(response.json())
    if response.status_code == 200:
        data = response.json()
        key = f'ISBN:{isbn}'
        if key in data:
            return data[key]
    return None

# 手動でISBN番号を入力する関数
def manual_isbn_input():
    isbn = st.text_input("ISBN番号を入力してください（例: 9784297108434）")
    st.write(f"入力されたISBN番号: {isbn}")  # デバッグ用
    if isbn:
        display_book_info(isbn)

In [28]:
import requests
get_book_info(9784297108434)

[{'onix': {'CollateralDetail': {}, 'RecordReference': '9784297108434', 'NotificationType': '03', 'ProductIdentifier': {'ProductIDType': '15', 'IDValue': '9784297108434'}, 'DescriptiveDetail': {'TitleDetail': {'TitleType': '01', 'TitleElement': {'TitleElementLevel': '01', 'TitleText': {'collationkey': 'カグル デ カツ データ ブンセキ ノ ギジュツ', 'content': 'Kaggleで勝つデータ分析の技術'}}}, 'Contributor': [{'SequenceNumber': '1', 'ContributorRole': [], 'PersonName': {'content': '門脇, 大輔', 'collationkey': 'カドワキ, ダイスケ'}}, {'SequenceNumber': '2', 'ContributorRole': [], 'PersonName': {'content': '阪田, 隆司', 'collationkey': 'サカタ, リュウジ'}}, {'SequenceNumber': '3', 'ContributorRole': [], 'PersonName': {'content': '保坂, 桂佑', 'collationkey': 'ホサカ, ケイスケ'}}]}, 'PublishingDetail': {'Imprint': {'ImprintName': '技術評論社'}, 'PublishingDate': [{'PublishingDateRole': '11', 'Date': '201910'}]}, 'ProductSupply': {'SupplyDetail': {'ProductAvailability': '99', 'Price': [{'PriceType': '01', 'CurrencyCode': 'JPY', 'PriceAmount': '3280'}]}}}, 'h